In [2]:
import re
import json
import tqdm
import random
from datasets import load_dataset
from transformers import AutoTokenizer
random.seed(101)

/home/gcilleru/ProphetNet/GENIE/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
dataset = load_dataset("recipe_nlg", data_dir="recipes/raw/dataset")

/home/gcilleru/ProphetNet/GENIE/.venv/lib/python3.8/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [3]:
with open("recipes/raw/masked_recipes.json", "r") as f:
    masked_recipes = json.load(f)

In [4]:
random.seed(101)
# split dataset into train, valid, test
random.shuffle(masked_recipes)
train = masked_recipes[:int(len(masked_recipes)*0.8)]
valid = masked_recipes[int(len(masked_recipes)*0.8):int(len(masked_recipes)*0.95)]
test = masked_recipes[int(len(masked_recipes)*0.95):]

In [7]:
train_ids = [recipe["id"] for recipe in train]
dev_ids = [recipe["id"] for recipe in valid]
test_ids = [recipe["id"] for recipe in test]

In [13]:
train_split = []
dev_split = []
test_split = []
for split, ids in zip([train_split, dev_split, test_split], [train_ids, dev_ids, test_ids]):
    for id in tqdm.tqdm(ids):
            split.append(dataset["train"][id])
                

100%|██████████| 98557/98557 [00:11<00:00, 8324.04it/s]


In [19]:
def build_recipe(sample):
    return sample["title"] + " Ingredients: " + " ".join(sample["ingredients"]) + " Directions: -" + " -".join(sample["directions"])
build_recipe(train_split[0])

'Bean, Pasta and Roasted Pepper Soup Ingredients: 1 tbsp. olive oil 2 large carrots 2 stalk celery 1 large onion 1 tbsp. chopped fresh parsley 3 clove garlic 1 can Swanson Chicken Broth 1 can black beans 1 can diced tomatoes 1/2 c. uncooked ditalini pasta 1/2 c. roasted sweet red pepper strips 1 tbsp. red wine vinegar Directions: -Heat oil in saucepot. -Add carrots, celery, onion, parsley and garlic and cook until tender. -Add broth, beans, tomatoes and pasta. -Heat to a boil. -Cover and cook 15 min. -or until pasta is done. -Add peppers and vinegar. -Heat through.'

In [21]:
# save data by writting it to a {split}.txt file in recipes/gpt2/data
for split, name in zip([train_split, dev_split, test_split], ["train", "dev", "test"]):
    with open(f"recipes/gpt2/data/{name}.txt", "w") as f:
        for recipe in split:
            f.write(build_recipe(recipe) + "\n")

In [23]:
train_path = 'recipes/gpt2/data/train.txt'
dev_path = 'recipes/gpt2/data/dev.txt'
test_path = 'recipes/gpt2/data/test.txt'

from torch.utils.data import Dataset
from transformers import GPT2Tokenizer

class TextDataset(Dataset):
    def __init__(self, file_path):
        self.file_path = file_path
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        self.inputs = []
        self.masks = []

    def read_data(self):
        with open(self.file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            for line in lines:
                encodings = self.tokenizer(line.strip(), truncation=True, max_length=512, padding="max_length")
                self.inputs.append(encodings['input_ids'])
                self.masks.append(encodings['attention_mask'])

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs[idx],
            'attention_mask': self.masks[idx]
        }


from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

train_dataset = TextDataset(train_path)
train_dataset.read_data()

dev_dataset = TextDataset(dev_path)
dev_dataset.read_data()


import torch
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir="./gpt2_finetuned", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 10000, # Number of update steps between two evaluations.
    save_steps=10000, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
)
    
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]), 'labels': torch.stack([f[1] for f in data])},
    train_dataset=train_dataset,
    eval_dataset=dev_dataset
)

Downloading: 100%|██████████| 548M/548M [00:05<00:00, 94.7MB/s] 


In [ ]:
# !CUDA_VISIBLE_DEVICES=3 python finetune_gpt2.py

In [1]:
from transformers import TextDataset, DataCollatorForLanguageModeling


/home/gcilleru/ProphetNet/GENIE/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
